In [1]:
import ee
import folium
import plotly.graph_objects as go
import plotly.io as pio

In [4]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='eo4c-nicosherpa')

#### General Functions and Data Loading

In [ ]:
# Function to Mask Clouds and Cloud Shadows in Landsat 8 Imagery
def cloudMask(image):
    cloudShadowBitmask = (1 << 3)
    cloudBitmask = (1 << 5)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloudShadowBitmask).eq(0).And(qa.bitwiseAnd(cloudBitmask).eq(0))
    return image.updateMask(mask)

# Function to apply scaling factors for Landsat 8 imagery
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# Import and preprocess Landsat 8 imagery
landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").map(applyScaleFactors).map(cloudMask)

# Function to calculate the average LST for a given period
def calculateAverageLST(startDate, endDate, area):
    period = landsat8.filterBounds(area).filterDate(startDate, endDate).median().clip(area)
    lst = period.select('ST_B10').subtract(273.15).rename('LST')
    return lst

#### Defining Location and Years

In [6]:
# Define the coordinates for Portland, Oregon
portland = ee.Geometry.Point([-122.6784, 45.5152])

# Define bounding boxes around Portland and Scappoose
portlandBBox = ee.Geometry.Rectangle([-123.0, 45.3, -122.3, 45.7])
scappooseBBox = ee.Geometry.Rectangle([-123.1, 45.7, -122.7, 45.9])

# Define example points within the Portland bounding box
examplePoints = [
    ee.Geometry.Point([-122.8057583, 45.486836]),  # Example Point 1
    ee.Geometry.Point([-122.6589563, 45.5214872]),  # Example Point 2
    ee.Geometry.Point([-122.5846527, 45.5555345])   # Example Point 3
]

In [ ]:
# Define the list of years to analyze
years = list(range(2013, 2024))

### Example Point Output and Averages of Portland and Scappoose over the years

#### Example Point Map and Chart Output

In [12]:
# Colors for the example points
colors = ['red', 'green', 'blue']

# Calculate average yearly LST for Scappoose
scappoose_avg_lst = {}
for year in years:
    startDate = ee.Date.fromYMD(year, 1, 1)
    endDate = ee.Date.fromYMD(year, 12, 31)
    scappooseLST = calculateAverageLST(startDate.format('YYYY-MM-dd').getInfo(), endDate.format('YYYY-MM-dd').getInfo(), scappooseBBox)
    scappooseMeanLST = scappooseLST.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=scappooseBBox,
        scale=30,
        maxPixels=1e9
    ).get('LST').getInfo()
    scappoose_avg_lst[year] = scappooseMeanLST

# Create an interactive chart for LST values
fig1 = go.Figure()
# Create an interactive chart for LST differences
fig2 = go.Figure()

for index, (point, color) in enumerate(zip(examplePoints, colors)):
    pixelValues = []
    differences = []
    for year in years:
        startDate = ee.Date.fromYMD(year, 1, 1)
        endDate = ee.Date.fromYMD(year, 12, 31)
        pixelLST = calculateAverageLST(startDate.format('YYYY-MM-dd').getInfo(), endDate.format('YYYY-MM-dd').getInfo(), point)
        value = pixelLST.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=30,
            maxPixels=1e9
        ).get('LST').getInfo()
        pixelValues.append([year, value])
        differences.append([year, value - scappoose_avg_lst[year]])
    
    years_list = [pv[0] for pv in pixelValues]
    lst_list = [pv[1] for pv in pixelValues]
    diff_list = [df[1] for df in differences]
    
    fig1.add_trace(go.Scatter(
        x=years_list, 
        y=lst_list, 
        mode='lines+markers', 
        name=f'', 
        line=dict(color=color),
        marker=dict(color=color),
        hovertemplate='Example Point ' + str(index + 1) + '<br>Year: %{x}<br>LST: %{y:.2f} °C'
    ))

    fig2.add_trace(go.Scatter(
        x=years_list, 
        y=diff_list, 
        mode='lines+markers', 
        name=f'', 
        line=dict(color=color),
        marker=dict(color=color),
        hovertemplate='Example Point ' + str(index + 1) + '<br>Year: %{x}<br>Difference: %{y:.2f} °C'
    ))


fig1.update_layout(
    title='LST Development Over Time for Example Points',
    xaxis_title='Year',
    yaxis_title='LST (°C)',
    legend_title='Example Points',
    template='plotly_white'
)

fig2.update_layout(
    title='LST Difference from Scappoose Average Over Time for Example Points',
    xaxis_title='Year',
    yaxis_title='Difference (°C)',
    legend_title='Example Points',
    template='plotly_white'
)

# Show the charts
fig1.show()
fig2.show()

# Save the interactive charts as HTML files
pio.write_html(fig1, file='examplePoints/LST_Development_Chart.html', auto_open=True)
pio.write_html(fig2, file='examplePoints/LST_Difference_Chart.html', auto_open=True)

# Create a map to show the example points
m = folium.Map(location=[45.5152, -122.6784], zoom_start=10)

# Add example points to the map
for index, point in enumerate(examplePoints):
    folium.Marker(
        location=[point.coordinates().get(1).getInfo(), point.coordinates().get(0).getInfo()],
        popup=f'Point {index + 1}',
        icon=folium.Icon(color=colors[index])
    ).add_to(m)

# Save the map as an HTML file
m.save('examplePoints/Example_Points_Map.html')


#### Printout of Average Portland and Scappoose values

In [8]:
# Calculate and display LST changes and print average LST for Portland and Scappoose
for year in years:
    startDate = ee.Date.fromYMD(year, 1, 1)
    endDate = ee.Date.fromYMD(year, 12, 31)

    # Calculate average LST for Scappoose
    scappooseLST = calculateAverageLST(startDate.format('YYYY-MM-dd').getInfo(), endDate.format('YYYY-MM-dd').getInfo(), scappooseBBox)
    scappooseMeanLST = scappooseLST.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=scappooseBBox,
        scale=30,
        maxPixels=1e9
    ).get('LST').getInfo()

    # Calculate average LST for Portland
    portlandLST = calculateAverageLST(startDate.format('YYYY-MM-dd').getInfo(), endDate.format('YYYY-MM-dd').getInfo(), portlandBBox)
    portlandMeanLST = portlandLST.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=portlandBBox,
        scale=30,
        maxPixels=1e9
    ).get('LST').getInfo()

    # Print the average LST for both Portland and Scappoose for each year
    print('Year:', year)
    print('Average LST for Portland in', year, ':', portlandMeanLST)
    print('Average LST for Scappoose in', year, ':', scappooseMeanLST)


Year: 2013
Average LST for Portland in 2013 : 24.294163099782775
Average LST for Scappoose in 2013 : 21.58200323120485
Year: 2014
Average LST for Portland in 2014 : 22.53575519252868
Average LST for Scappoose in 2014 : 15.313030403090027
Year: 2015
Average LST for Portland in 2015 : 26.7173193996444
Average LST for Scappoose in 2015 : 22.734791531529982
Year: 2016
Average LST for Portland in 2016 : 25.383447676305153
Average LST for Scappoose in 2016 : 21.230963929227347
Year: 2017
Average LST for Portland in 2017 : 24.648463657535988
Average LST for Scappoose in 2017 : 23.63177855866009
Year: 2018
Average LST for Portland in 2018 : 23.015259866682992
Average LST for Scappoose in 2018 : 21.16178060502277
Year: 2019
Average LST for Portland in 2019 : 24.087193729283456
Average LST for Scappoose in 2019 : 19.011342877527944
Year: 2020
Average LST for Portland in 2020 : 21.341494285625814
Average LST for Scappoose in 2020 : 18.25277692382604
Year: 2021
Average LST for Portland in 2021 : 2

### Map Outputs

#### City and Suburb Boxes

In [15]:
# Create a folium map centered on Portland
map_center = [45.5152, -122.6784]
m = folium.Map(location=map_center, zoom_start=10)

# Function to add bounding boxes to the folium map
def add_bounding_box(map_object, bbox, color, name):
    folium.GeoJson(
        bbox.getInfo(),
        style_function=lambda feature: {
            'color': color,
            'weight': 2,
            'fillOpacity': 0
        },
        name=name
    ).add_to(map_object)

# Add the bounding boxes to the map
add_bounding_box(m, portlandBBox, 'blue', 'Portland Bounding Box')
add_bounding_box(m, scappooseBBox, 'green', 'Scappoose Bounding Box')

# Add layer control to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save('mapLST/Bounding_Boxes_Map.html')

#### Map Output Yearly Average

In [13]:
# Create a folium map centered on Portland
map_center = [45.5152, -122.6784]
m = folium.Map(location=map_center, zoom_start=10)

# Function to add Earth Engine layer to the folium map
def add_ee_layer(map_object, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(map_object)

# Loop through each year to calculate and display LST changes
for year in years:
    startDate = ee.Date.fromYMD(year, 1, 1)
    endDate = ee.Date.fromYMD(year, 12, 31)

    # Calculate average LST for Scappoose
    scappooseLST = calculateAverageLST(startDate.format('YYYY-MM-dd').getInfo(), endDate.format('YYYY-MM-dd').getInfo(), scappooseBBox)
    scappooseMeanLST = scappooseLST.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=scappooseBBox,
        scale=30,
        maxPixels=1e9
    ).get('LST').getInfo()

    # Calculate average LST for Portland
    portlandLST = calculateAverageLST(startDate.format('YYYY-MM-dd').getInfo(), endDate.format('YYYY-MM-dd').getInfo(), portlandBBox)
    portlandMeanLST = portlandLST.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=portlandBBox,
        scale=30,
        maxPixels=1e9
    ).get('LST').getInfo()

    # Calculate deviations from the Scappoose mean
    deviation = portlandLST.subtract(scappooseMeanLST)

    # Visualization parameters for deviations
    visParams = {
        'min': -20,
        'max': 20,
        'palette': [
            '0000FF', '5050FF', 'A0A0FF', 'FFFFFF', 'FFA0A0', 'FF5050', 'FF0000'
        ]
    }

    # Add the deviation layer to the map
    add_ee_layer(m, deviation, visParams, 'LST Deviation ' + str(year))

# Add layer control to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save('mapLST/LST_Deviation_Map.html')

#### Map Output Monthly Average

In [14]:
# Function to calculate the average LST for a given month across multiple years
def calculateMonthlyAverageLST(month, startYear, endYear, area):
    monthlyLST = ee.ImageCollection(landsat8.filter(ee.Filter.calendarRange(month, month, 'month'))
        .filter(ee.Filter.calendarRange(startYear, endYear, 'year'))
        .filterBounds(area)
        .median()
        .clip(area)
        .select('ST_B10')
        .subtract(273.15)
        .rename('LST'))
    
    return monthlyLST.mean()

# Define the list of months to analyze
months = list(range(1, 13))

# Create a folium map centered on Portland
map_center = [45.5152, -122.6784]
m = folium.Map(location=map_center, zoom_start=10)

# Function to add Earth Engine layer to the folium map
def add_ee_layer(map_object, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(map_object)

# Loop through each month to calculate and display LST changes
for month in months:
    # Calculate average LST for Scappoose for the month
    scappooseLST = calculateMonthlyAverageLST(month, 2013, 2023, scappooseBBox)
    scappooseMeanLST = scappooseLST.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=scappooseBBox,
        scale=30,
        maxPixels=1e9
    ).get('LST').getInfo()

    # Calculate average LST for Portland for the month
    portlandLST = calculateMonthlyAverageLST(month, 2013, 2023, portlandBBox)
    portlandMeanLST = portlandLST.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=portlandBBox,
        scale=30,
        maxPixels=1e9
    ).get('LST').getInfo()

    # Calculate deviations from the Scappoose mean
    deviation = portlandLST.subtract(scappooseMeanLST)

    # Visualization parameters for deviations
    visParams = {
        'min': -20,
        'max': 20,
        'palette': [
            '0000FF', '5050FF', 'A0A0FF', 'FFFFFF', 'FFA0A0', 'FF5050', 'FF0000'
        ]
    }

    # Add the deviation layer to the map
    add_ee_layer(m, deviation, visParams, f'LST Deviation Month {month}')

# Add layer control to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save('mapLST/LST_Deviation_Map_Monthly.html')
